In [5]:
%tensorflow_version 1.x   #Using tensorflow 1.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':      #use GPU for fast process
  raise SystemError('GPU device not found')
#print('Found GPU at: {}'.format(device_name))

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.x   #Using tensorflow 1.x`. This will be interpreted as: `1.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


In [6]:
print(tf.__version__)

1.15.2


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import requests  
file_url = "http://nlp.stanford.edu/data/glove.6B.zip"          #Download glove here is the link
    
r = requests.get(file_url, stream = True)  
  
with open("glove.6B.zip", "wb") as file:  
    for block in r.iter_content(chunk_size = 1024): 
         if block:  
             file.write(block) 

In [9]:
!apt install unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 59 not upgraded.


In [10]:
cd '/content/go'      #downloaded in server but not in my drive it is fast to access

[Errno 2] No such file or directory: '/content/go #downloaded in server but not in my drive it is fast to access'
/content


In [11]:
!unzip  '/content/glove.6B.zip' -d go

Archive:  /content/glove.6B.zip
replace go/glove.6B.50d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: go/glove.6B.50d.txt     
replace go/glove.6B.100d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: go/glove.6B.100d.txt    y

replace go/glove.6B.200d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: go/glove.6B.200d.txt    y

replace go/glove.6B.300d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: go/glove.6B.300d.txt    y
y



In [12]:
'''import numpy as np
np.random.seed(42)
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import keras
from keras.models import Model
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D,GlobalAveragePooling1D
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
import warnings
warnings.filterwarnings('ignore')'''
import numpy as np
import pandas as pd
import re
import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from keras.models import Model
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [212]:
EMBEDDING_FILE = '/content/go/glove.6B.300d.txt'      
train = pd.read_csv('/content/drive/My Drive/Hate Speech/english_dataset.tsv', sep = '\t', encoding="latin-1")
test = pd.read_csv('/content/drive/My Drive/Hate Speech/hasoc2019_en_test-2919.tsv', sep = '\t', encoding="latin-1")

In [213]:
train=train[train.task_3!="NONE"]       #drop none
test=test[test.task_3!="NONE"]
train=train.reset_index(drop=True)
test=test.reset_index(drop=True)
X_train = train["text"]
y_train = train["task_3"]                     #For task 3         
X_test = test["text"]
y_test = test["task_3"]

In [214]:
y_train

0       TIN
1       TIN
2       TIN
3       TIN
4       TIN
       ... 
2256    UNT
2257    UNT
2258    TIN
2259    UNT
2260    UNT
Name: task_3, Length: 2261, dtype: object

In [215]:
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [216]:
#Remove stopwords from the text
stop_words = set(stopwords.words('english')) 
ix=0
for i in X_train:
  word_tokens = (i[0].split())  
  for w in word_tokens: 
    if w not in stop_words: 
        X_train[ix]=X_train[ix]+(w) 
  ix=ix+1
ix=0
for i in X_test:
  word_tokens = (i[0].split())  
  for w in word_tokens: 
    if w not in stop_words: 
        X_test[ix]= X_test[ix]+(w) 
  ix=ix+1

In [217]:
#remove punctuations
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_text(x):
    x = str(x)
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, '')
    return x

In [218]:
ix=0
for i in X_train:
  X_train[ix]=clean_text(i)
  ix=ix+1
ix=0
for i in X_test:
  X_test[ix]=clean_text(i)
  ix=ix+1

In [219]:
#clean the numbers
def clean_numbers(x):
    if bool(re.search(r'\d', x)):
        x = re.sub('[0-9]{5,}', '#####', x)
        x = re.sub('[0-9]{4}', '####', x)
        x = re.sub('[0-9]{3}', '###', x)
        x = re.sub('[0-9]{2}', '##', x)
    return x

In [220]:
ix=0
for i in X_train:
  X_train[ix]=clean_numbers(i)
  ix=ix+1
ix=0
for i in X_test:
  X_test[ix]=clean_numbers(i)
  ix=ix+1

In [221]:
#remove contractions
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

def _get_contractions(contraction_dict):
    contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
    return contraction_dict, contraction_re

contractions, contractions_re = _get_contractions(contraction_dict)

def replace_contractions(text):
    def replace(match):
        return contractions[match.group(0)]
    return contractions_re.sub(replace, text)


In [222]:
ix=0
for i in X_train:
  X_train[ix]=replace_contractions(i)
  ix=ix+1
ix=0
for i in X_test:
  X_test[ix]=replace_contractions(i)
  ix=ix+1

In [223]:
#Stemming
'''Stemming is the process of converting words to their base forms using crude Heuristic rules.
   For example, one rule could be to remove ’s’ from the end of any word, so that ‘cats’ becomes ‘cat’. 
   or another rule could be to replace ‘ies’ with ‘i’ so that ‘ponies becomes ‘poni’.'''
from nltk.stem import  SnowballStemmer
from nltk.tokenize.toktok import ToktokTokenizer
def stem_text(text):
    tokenizer = ToktokTokenizer()
    stemmer = SnowballStemmer('english')
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(tokens)

In [224]:
ix=0
for i in X_train:
  X_train[ix]=stem_text(i)
  ix=ix+1
ix=0
for i in X_test:
  X_test[ix]=stem_text(i)
  ix=ix+1

In [225]:
type(X_train)

pandas.core.series.Series

In [226]:
#After removing or cleaning the text
max_features = 11118  #No. of different words in total text
maxlen = 70           #Padding length
embed_size = 300      #No. of dimensions of glove

threshold = 0.35

tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train) + list(X_test))
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [227]:
y_train = y_train.map({'TIN':0, 'UNT': 1})    #Map accordingly 
y_test = y_test.map({'TIN':0, 'UNT': 1})

In [228]:
y_train=keras.utils.to_categorical(y_train, num_classes=2)  #one hot encoding
y_test=keras.utils.to_categorical(y_test, num_classes=2)


In [229]:
y=[]
for i in y_train:
 y.append(np.argmax(i))

In [230]:
def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE))

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [231]:
num_filters = 42
filter_sizes = [1,2,3,5]  #ngrams

In [232]:
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D,LSTM,SimpleRNN,GRU
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.optimizers import Adam
from keras.models import Model
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers


from keras.layers import *
from keras.models import *
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.initializers import *
from keras.optimizers import *
import keras.backend as K
from keras.callbacks import *

def model_train_cv(x_train,y_train,nfold,model_obj):
    splits = list(StratifiedKFold(n_splits=nfold, shuffle=True, random_state=SEED).split(x_train, y_train))
    x_train = x_train
    y_train = np.array(y_train)
    # matrix for the out-of-fold predictions
    train_oof_preds = np.zeros((x_train.shape[0]))
    for i, (train_idx, valid_idx) in enumerate(splits):
        print(f'Fold {i + 1}')
        x_train_fold = x_train[train_idx.astype(int)]
        y_train_fold = y_train[train_idx.astype(int)]
        x_val_fold = x_train[valid_idx.astype(int)]
        y_val_fold = y_train[valid_idx.astype(int)]
        # Changed it here a little bit since the custom attention layer is not getting deepcopy
        clf = model_obj
        clf.load_weights('model.h5')
        clf.fit(x_train_fold, y_train_fold, batch_size=512, epochs=5, validation_data=(x_val_fold, y_val_fold))
        
        valid_preds_fold = clf.predict(x_val_fold)[:,0]

        # storing OOF predictions
        train_oof_preds[valid_idx] = valid_preds_fold
    return train_oof_preds

In [233]:
#model of bi-lstm with attention
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
    

class AttentionWithContext(Layer):
    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight(shape=(input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]


def model_lstm_atten(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x = Bidirectional(CuDNNLSTM(128, return_sequences=True))(x)
    x = Bidirectional(CuDNNLSTM(64, return_sequences=True))(x)
    x = AttentionWithContext()(x)
    x = Dense(64, activation="relu")(x)
    x = Dense(2, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [234]:
#model of lstm with attention
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
    

class AttentionWithContext(Layer):
    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight(shape=(input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]


def model_slstm_atten(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    x = AttentionWithContext()(x)
    x = Dense(64, activation="relu")(x)
    x = Dense(2, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [235]:
#model of gru with attention
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
    

class AttentionWithContext(Layer):
    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight(shape=(input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]


def model_gru_atten(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x = Bidirectional(GRU(128, return_sequences=True))(x)
    x = Bidirectional(GRU(64, return_sequences=True))(x)
    x = AttentionWithContext()(x)
    x = Dense(64, activation="relu")(x)
    x = Dense(2, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [236]:
#model of Simple RNN with attention
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
    

class AttentionWithContext(Layer):
    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight(shape=(input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]


def model_rnn_atten(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x = Bidirectional(SimpleRNN(128, return_sequences=True))(x)
    x = Bidirectional(SimpleRNN(64, return_sequences=True))(x)
    x = AttentionWithContext()(x)
    x = Dense(64, activation="relu")(x)
    x = Dense(2, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [237]:
print(tf.__version__)

1.15.2


In [238]:
ls_model = model_lstm_atten(embedding_matrix) #model of bi-lstm with attention

In [239]:
sls_model = model_slstm_atten(embedding_matrix) #model of lstm with attention

In [240]:
gru_model = model_gru_atten(embedding_matrix) #model of gru with attention

In [241]:
rnn_model = model_rnn_atten(embedding_matrix) #model of rnn with attention

In [242]:
batch_size = 256
epochs = 10

X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.95,
                                              random_state=42) #split this into train and validation

In [243]:
ls_hist = ls_model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs,
                 validation_data=(X_val, y_val),
                  verbose=2)      #Train bi-lstm with attention

Train on 2147 samples, validate on 114 samples
Epoch 1/10
 - 2s - loss: 0.4593 - accuracy: 0.9017 - val_loss: 0.3501 - val_accuracy: 0.8947
Epoch 2/10
 - 0s - loss: 0.3313 - accuracy: 0.9031 - val_loss: 0.3351 - val_accuracy: 0.8947
Epoch 3/10
 - 0s - loss: 0.3249 - accuracy: 0.9031 - val_loss: 0.3399 - val_accuracy: 0.8947
Epoch 4/10
 - 0s - loss: 0.3210 - accuracy: 0.9031 - val_loss: 0.3376 - val_accuracy: 0.8947
Epoch 5/10
 - 0s - loss: 0.3185 - accuracy: 0.9031 - val_loss: 0.3397 - val_accuracy: 0.8947
Epoch 6/10
 - 0s - loss: 0.3184 - accuracy: 0.9031 - val_loss: 0.3373 - val_accuracy: 0.8947
Epoch 7/10
 - 0s - loss: 0.3187 - accuracy: 0.9031 - val_loss: 0.3382 - val_accuracy: 0.8947
Epoch 8/10
 - 0s - loss: 0.3184 - accuracy: 0.9031 - val_loss: 0.3371 - val_accuracy: 0.8947
Epoch 9/10
 - 0s - loss: 0.3180 - accuracy: 0.9031 - val_loss: 0.3370 - val_accuracy: 0.8947
Epoch 10/10
 - 0s - loss: 0.3175 - accuracy: 0.9031 - val_loss: 0.3354 - val_accuracy: 0.8947


In [244]:
sls_hist = sls_model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs,
                 validation_data=(X_val, y_val),
                  verbose=2)    #Train lstm with attention

Train on 2147 samples, validate on 114 samples
Epoch 1/10
 - 5s - loss: 0.4385 - accuracy: 0.8933 - val_loss: 0.3525 - val_accuracy: 0.8947
Epoch 2/10
 - 2s - loss: 0.3291 - accuracy: 0.9031 - val_loss: 0.3362 - val_accuracy: 0.8947
Epoch 3/10
 - 2s - loss: 0.3246 - accuracy: 0.9031 - val_loss: 0.3365 - val_accuracy: 0.8947
Epoch 4/10
 - 2s - loss: 0.3179 - accuracy: 0.9031 - val_loss: 0.3355 - val_accuracy: 0.8947
Epoch 5/10
 - 2s - loss: 0.3180 - accuracy: 0.9031 - val_loss: 0.3349 - val_accuracy: 0.8947
Epoch 6/10
 - 2s - loss: 0.3160 - accuracy: 0.9031 - val_loss: 0.3331 - val_accuracy: 0.8947
Epoch 7/10
 - 2s - loss: 0.3170 - accuracy: 0.9031 - val_loss: 0.3342 - val_accuracy: 0.8947
Epoch 8/10
 - 2s - loss: 0.3150 - accuracy: 0.9031 - val_loss: 0.3290 - val_accuracy: 0.8947
Epoch 9/10
 - 2s - loss: 0.3110 - accuracy: 0.9031 - val_loss: 0.3181 - val_accuracy: 0.8947
Epoch 10/10
 - 2s - loss: 0.2908 - accuracy: 0.9031 - val_loss: 0.2983 - val_accuracy: 0.8947


In [245]:
gru_hist = gru_model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs,
                 validation_data=(X_val, y_val),
                  verbose=2)      #Train GRU with attention

Train on 2147 samples, validate on 114 samples
Epoch 1/10
 - 6s - loss: 0.4445 - accuracy: 0.8812 - val_loss: 0.3531 - val_accuracy: 0.8947
Epoch 2/10
 - 3s - loss: 0.3349 - accuracy: 0.9031 - val_loss: 0.3437 - val_accuracy: 0.8947
Epoch 3/10
 - 3s - loss: 0.3211 - accuracy: 0.9031 - val_loss: 0.3371 - val_accuracy: 0.8947
Epoch 4/10
 - 3s - loss: 0.3131 - accuracy: 0.9031 - val_loss: 0.3340 - val_accuracy: 0.8947
Epoch 5/10
 - 3s - loss: 0.3053 - accuracy: 0.9031 - val_loss: 0.3261 - val_accuracy: 0.8947
Epoch 6/10
 - 3s - loss: 0.2975 - accuracy: 0.9031 - val_loss: 0.3167 - val_accuracy: 0.8947
Epoch 7/10
 - 3s - loss: 0.2850 - accuracy: 0.9031 - val_loss: 0.3105 - val_accuracy: 0.8947
Epoch 8/10
 - 3s - loss: 0.2602 - accuracy: 0.9036 - val_loss: 0.3034 - val_accuracy: 0.8860
Epoch 9/10
 - 3s - loss: 0.2349 - accuracy: 0.9087 - val_loss: 0.3131 - val_accuracy: 0.8947
Epoch 10/10
 - 3s - loss: 0.2114 - accuracy: 0.9148 - val_loss: 0.3339 - val_accuracy: 0.8947


In [246]:
rnn_hist = rnn_model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs,
                 validation_data=(X_val, y_val),
                  verbose=2)      #Train RNN with attention

Train on 2147 samples, validate on 114 samples
Epoch 1/10
 - 4s - loss: 0.3952 - accuracy: 0.8975 - val_loss: 0.3471 - val_accuracy: 0.8947
Epoch 2/10
 - 1s - loss: 0.3258 - accuracy: 0.9031 - val_loss: 0.3371 - val_accuracy: 0.8947
Epoch 3/10
 - 1s - loss: 0.3092 - accuracy: 0.9031 - val_loss: 0.3364 - val_accuracy: 0.8947
Epoch 4/10
 - 1s - loss: 0.2851 - accuracy: 0.9041 - val_loss: 0.3307 - val_accuracy: 0.8947
Epoch 5/10
 - 1s - loss: 0.2580 - accuracy: 0.9041 - val_loss: 0.3389 - val_accuracy: 0.8947
Epoch 6/10
 - 1s - loss: 0.2142 - accuracy: 0.9045 - val_loss: 0.3331 - val_accuracy: 0.9035
Epoch 7/10
 - 1s - loss: 0.1630 - accuracy: 0.9092 - val_loss: 0.3710 - val_accuracy: 0.9035
Epoch 8/10
 - 1s - loss: 0.1280 - accuracy: 0.9404 - val_loss: 0.3693 - val_accuracy: 0.8509
Epoch 9/10
 - 1s - loss: 0.0924 - accuracy: 0.9702 - val_loss: 0.4435 - val_accuracy: 0.8509
Epoch 10/10
 - 1s - loss: 0.0560 - accuracy: 0.9823 - val_loss: 0.5714 - val_accuracy: 0.8684


In [247]:
from sklearn.metrics import classification_report

In [248]:
y_t=[]
for i in y_test:
  y_t.append(np.argmax(i))    #Using argmax to find the index of one hot encoding

In [249]:
y_ls_pred = ls_model.predict(x_test)      #predict y_test by bi-lstm with attention
y_ls_p=[]
for i in y_ls_pred:
  y_ls_p.append(np.argmax(i))


In [250]:
print('Model = Attention Bidirectonal LSTM\n')  
print(classification_report(y_t, y_ls_p))

Model = Attention Bidirectonal LSTM

              precision    recall  f1-score   support

           0       0.85      1.00      0.92       245
           1       0.00      0.00      0.00        43

    accuracy                           0.85       288
   macro avg       0.43      0.50      0.46       288
weighted avg       0.72      0.85      0.78       288



In [251]:
y_sls_pred = sls_model.predict(x_test)  #predict y_test by lstm with attention
y_sls_p=[]
for i in y_sls_pred:
  y_sls_p.append(np.argmax(i))

In [252]:
print('Model = Attention LSTM\n')
print(classification_report(y_t, y_sls_p))

Model = Attention LSTM

              precision    recall  f1-score   support

           0       0.85      1.00      0.92       245
           1       0.00      0.00      0.00        43

    accuracy                           0.85       288
   macro avg       0.43      0.50      0.46       288
weighted avg       0.72      0.85      0.78       288



In [253]:
y_gru_pred = gru_model.predict(x_test)    #predict y_test by GRU with attention
y_gru_p=[]
for i in y_gru_pred:
  y_gru_p.append(np.argmax(i))

In [254]:
print('Model = Attention GRU\n')
print(classification_report(y_t, y_gru_p))

Model = Attention GRU

              precision    recall  f1-score   support

           0       0.85      0.92      0.88       245
           1       0.13      0.07      0.09        43

    accuracy                           0.79       288
   macro avg       0.49      0.49      0.49       288
weighted avg       0.74      0.79      0.76       288



In [255]:
y_rnn_pred = rnn_model.predict(x_test)    #predict y_test by RNN with attention
y_rnn_p=[]
for i in y_rnn_pred:
  y_rnn_p.append(np.argmax(i))

In [256]:
print('Model = Attention SimpleRNN\n')
print(classification_report(y_t, y_rnn_p))

Model = Attention SimpleRNN

              precision    recall  f1-score   support

           0       0.85      0.94      0.89       245
           1       0.17      0.07      0.10        43

    accuracy                           0.81       288
   macro avg       0.51      0.50      0.50       288
weighted avg       0.75      0.81      0.77       288

